In [ ]:
# it seems the tcgplayer api is no longer available for new people, so I will have to scrape some data on my own

In [22]:
from selenium import webdriver
import time
import math
import re
import itertools
import pandas as pd

In [30]:

# Get button you are going to click by its id ( also you could us find_element_by_css_selector to get element by css selector)
# time.sleep(2.5)
# button_element = driver.find_element_by_class_name('modal__activator')
# button_element.click()


'<html lang="en"><head><meta charset="utf-8"><meta http-equiv="Content-Type" content="text/html; charset=utf-8"><meta http-equiv="X-UA-Compatible" content="IE=edge"><meta name="viewport" content="width=device-width,initial-scale=1,viewport-fit=cover"><meta name="version-build" content="1834"><meta name="version-hash" content="5524bd4948e7f1636f14e3a2cf43331bee4dbb7b"><meta name="version-name" content="origin/hotfix-rel-795"><meta name="facebook-domain-verification" content="ujhbh9jn9ccwotzunt4vehh5l26w5i"><script>;window.NREUM||(NREUM={});NREUM.init={distributed_tracing:{enabled:true},privacy:{cookies_enabled:true},ajax:{deny_list:["bam-cell.nr-data.net"]}};\n      \n      ;NREUM.loader_config={accountID:"2099792",trustKey:"2099792",agentID:"153290625",licenseKey:"a7f40de1df",applicationID:"153290625"};\n      ;NREUM.info={beacon:"bam-cell.nr-data.net",errorBeacon:"bam-cell.nr-data.net",licenseKey:"a7f40de1df",applicationID:"153290625",sa:1};\n      (()=>{"use strict";var e,t,r={5763:(

In [ ]:
# section_class = "latest-sales sales-history-snapshot__latest-sales"

In [ ]:
# the text we're interested in is tbody then the dates and price that follows

In [2]:
# this is how to obtain a title
def get_title(html_source, item_number):
    regex_title = re.compile('{}(.*){}'.format(re.escape('search-result__title">'), re.escape('</span><div class="inventory"')))
    html_title = re.findall('search-result__title">[^<]+</span><div class="inventory"',html_source)
    title = regex_title.findall(html_title[item_number])
    return(title)

In [15]:
def get_table_data(html, title):
    regex_table = re.compile('{}(.*){}'.format(re.escape('tbody'), re.escape('tbody')))
    html_table_1 = regex_table.findall(htmls[0])
    dates = re.findall('\d{1,2}/\d{1,2} to \d{1,2}/\d{1,2}',html_table_1[0])
    prices = re.findall('\$\d+(?:\.\d+)?',html_table_1[0])
    titles = title * len(dates)
    return titles, dates, prices

In [16]:
url = 'https://www.tcgplayer.com/search/pokemon/sv-scarlet-and-violet-151?productLineName=pokemon&setName=sv-scarlet-and-violet-151&view=grid&page=1'
driver = webdriver.Chrome(executable_path=r"C:\Users\steve\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe")
driver.get(url)
# driver.implicitly_wait(10)
time.sleep(3)
html_source = driver.page_source


# how to find number of search results
regex = re.compile('{}(.*){}'.format(re.escape('search-result-count'), re.escape(' results')))
shortened_html = regex.findall(html_source)
num_results = re.findall('^.*>([0-9]+)$',shortened_html[0])
num_results = int(num_results[0])
num_pages = math.ceil(num_results/24)


htmls = []
titles = []
dates = []
prices = []

for h in range(num_pages):
    url = url[:-1] + str(h+1)
    driver.get(url)
    time.sleep(10)
    item_titles = driver.find_elements_by_class_name('search-result__title')
    for i in range(len(item_titles)):
        html_source = driver.page_source
        title = get_title(html_source, i)
        
        item_titles = driver.find_elements_by_class_name('search-result__title')
        print(h+1, i, len(item_titles))
        item_titles[i].click()
        time.sleep(10)
        html = driver.page_source
        htmls.append(html)
        
        title, date, price = get_table_data(html, title)
        titles.append(title)
        dates.append(date)
        prices.append(price)
        
        driver.back()
        time.sleep(10)


1 0 24
1 1 24
1 2 24
1 3 24
1 4 24
1 5 24
1 6 24
1 7 24
1 8 24
1 9 24
1 10 24
1 11 24
1 12 24
1 13 24
1 14 24
1 15 24
1 16 24
1 17 24
1 18 24
1 19 24
1 20 24
1 21 24
1 22 24
1 23 24
2 0 24
2 1 24
2 2 24
2 3 24
2 4 24
2 5 24
2 6 24
2 7 24
2 8 24
2 9 24
2 10 24
2 11 24
2 12 24
2 13 24
2 14 24
2 15 24
2 16 24
2 17 24
2 18 24
2 19 24
2 20 24
2 21 24
2 22 24
2 23 24
3 0 24
3 1 24
3 2 24
3 3 24
3 4 24
3 5 24
3 6 24
3 7 24
3 8 24
3 9 24
3 10 24
3 11 24
3 12 24
3 13 24
3 14 24
3 15 24
3 16 24
3 17 24
3 18 24
3 19 24
3 20 24
3 21 24
3 22 24
3 23 24
4 0 24
4 1 24
4 2 24
4 3 24
4 4 24
4 5 24
4 6 24
4 7 24
4 8 24
4 9 24
4 10 24
4 11 24
4 12 24
4 13 24
4 14 24
4 15 24
4 16 24
4 17 24
4 18 24
4 19 24
4 20 24
4 21 24
4 22 24
4 23 24
5 0 24
5 1 24
5 2 24
5 3 24
5 4 24
5 5 24
5 6 24
5 7 24
5 8 24
5 9 24
5 10 24
5 11 24
5 12 24
5 13 24
5 14 24
5 15 24
5 16 24
5 17 24
5 18 24
5 19 24
5 20 24
5 21 24
5 22 24
5 23 24
6 0 24
6 1 24
6 2 24
6 3 24
6 4 24
6 5 24
6 6 24
6 7 24
6 8 24
6 9 24
6 10 24
6 11 24
6 12

In [17]:
len(titles), len(dates), len(prices)

(230, 230, 230)

In [18]:
titles = list(itertools.chain(*titles))
dates = list(itertools.chain(*dates))
prices = list(itertools.chain(*prices))

In [19]:
len(titles), len(dates), len(prices)

(6900, 6900, 6900)

In [31]:
market_prices = pd.DataFrame([titles, dates, prices]).T
market_prices = market_prices.rename(columns={0: 'Product', 1:'Date Range', 2: 'Market Price'})
market_prices

,Product,Date Range,Market Price
0,Code Card - 151 Booster Pack,7/14 to 7/16,$0.00
1,Code Card - 151 Booster Pack,7/17 to 7/19,$0.00
2,Code Card - 151 Booster Pack,7/20 to 7/22,$0.00
3,Code Card - 151 Booster Pack,7/23 to 7/25,$0.00
4,Code Card - 151 Booster Pack,7/26 to 7/28,$0.00
...,...,...,...
6895,151 Booster Bundle Display Case,9/27 to 9/29,$0.13
6896,151 Booster Bundle Display Case,9/30 to 10/2,$0.12
6897,151 Booster Bundle Display Case,10/3 to 10/5,$0.12
6898,151 Booster Bundle Display Case,10/6 to 10/8,$0.10


In [33]:
market_prices.to_csv('Pokemon_151.csv')

In [20]:
dates

['7/14 to 7/16',
 '7/17 to 7/19',
 '7/20 to 7/22',
 '7/23 to 7/25',
 '7/26 to 7/28',
 '7/29 to 7/31',
 '8/1 to 8/3',
 '8/4 to 8/6',
 '8/7 to 8/9',
 '8/10 to 8/12',
 '8/13 to 8/15',
 '8/16 to 8/18',
 '8/19 to 8/21',
 '8/22 to 8/24',
 '8/25 to 8/27',
 '8/28 to 8/30',
 '8/31 to 9/2',
 '9/3 to 9/5',
 '9/6 to 9/8',
 '9/9 to 9/11',
 '9/12 to 9/14',
 '9/15 to 9/17',
 '9/18 to 9/20',
 '9/21 to 9/23',
 '9/24 to 9/26',
 '9/27 to 9/29',
 '9/30 to 10/2',
 '10/3 to 10/5',
 '10/6 to 10/8',
 '10/9 to 10/11',
 '7/14 to 7/16',
 '7/17 to 7/19',
 '7/20 to 7/22',
 '7/23 to 7/25',
 '7/26 to 7/28',
 '7/29 to 7/31',
 '8/1 to 8/3',
 '8/4 to 8/6',
 '8/7 to 8/9',
 '8/10 to 8/12',
 '8/13 to 8/15',
 '8/16 to 8/18',
 '8/19 to 8/21',
 '8/22 to 8/24',
 '8/25 to 8/27',
 '8/28 to 8/30',
 '8/31 to 9/2',
 '9/3 to 9/5',
 '9/6 to 9/8',
 '9/9 to 9/11',
 '9/12 to 9/14',
 '9/15 to 9/17',
 '9/18 to 9/20',
 '9/21 to 9/23',
 '9/24 to 9/26',
 '9/27 to 9/29',
 '9/30 to 10/2',
 '10/3 to 10/5',
 '10/6 to 10/8',
 '10/9 to 10/11',
 '

In [99]:
len(htmls)
# len equal to the total number of search items!

230

In [103]:
html_table_1

[' data-v-d8f9e48a=""><tr data-v-d8f9e48a=""><td data-v-d8f9e48a="">7/11 to 7/13</td><td data-v-d8f9e48a="">$0.00</td></tr><tr data-v-d8f9e48a=""><td data-v-d8f9e48a="">7/14 to 7/16</td><td data-v-d8f9e48a="">$0.00</td></tr><tr data-v-d8f9e48a=""><td data-v-d8f9e48a="">7/17 to 7/19</td><td data-v-d8f9e48a="">$0.00</td></tr><tr data-v-d8f9e48a=""><td data-v-d8f9e48a="">7/20 to 7/22</td><td data-v-d8f9e48a="">$0.00</td></tr><tr data-v-d8f9e48a=""><td data-v-d8f9e48a="">7/23 to 7/25</td><td data-v-d8f9e48a="">$0.00</td></tr><tr data-v-d8f9e48a=""><td data-v-d8f9e48a="">7/26 to 7/28</td><td data-v-d8f9e48a="">$0.00</td></tr><tr data-v-d8f9e48a=""><td data-v-d8f9e48a="">7/29 to 7/31</td><td data-v-d8f9e48a="">$0.00</td></tr><tr data-v-d8f9e48a=""><td data-v-d8f9e48a="">8/1 to 8/3</td><td data-v-d8f9e48a="">$0.00</td></tr><tr data-v-d8f9e48a=""><td data-v-d8f9e48a="">8/4 to 8/6</td><td data-v-d8f9e48a="">$0.00</td></tr><tr data-v-d8f9e48a=""><td data-v-d8f9e48a="">8/7 to 8/9</td><td data-v-d

In [100]:
htmls[0]

'<html lang="en"><head><script type="text/javascript" src="https://api.getdrip.com/client/track?url=https%3A%2F%2Fwww.tcgplayer.com%2Fproduct%2F517267%2Fpokemon-sv-scarlet-and-violet-151-code-card-151-booster-pack%3Fxid%3Dpi5142b99c-d2a4-4f75-9f3d-3453e6bfdf8d%26page%3D1%26Language%3DEnglish&amp;visitor_uuid=5db0b7608d30411aacbd1d8140a9ae23&amp;application=MPFE&amp;domain=www.tcgplayer.com&amp;product_id=517267&amp;product_name=Code%20Card%20-%20151%20Booster%20Pack&amp;seller_spotlight_quantity=undefined&amp;product_line=Pokemon&amp;set_name=SV%3A%20Scarlet%20and%20Violet%20151&amp;is_sealed=false&amp;is_singles=true&amp;total_listings=148&amp;total_sellers=146&amp;is_near_mint_available=&amp;is_lightly_played_available=&amp;is_moderately_played_available=&amp;seller_key=&amp;seller_name=&amp;new_pd_group=true&amp;_action=Product%20Details%20Viewed&amp;drip_account_id=4160913&amp;callback=Drip_840363928"></script><script type="text/javascript" src="https://api.getdrip.com/client/track

In [57]:
html_source

'<html lang="en"><head><script type="text/javascript" src="https://api.getdrip.com/client/track?url=https%3A%2F%2Fwww.tcgplayer.com%2Fsearch%2Fpokemon%2Fsv-scarlet-and-violet-151%3FproductLineName%3Dpokemon%26setName%3Dsv-scarlet-and-violet-151%26view%3Dgrid%26page%3D1&amp;visitor_uuid=783eb89011df4c139816a25e56356e3b&amp;results[]=%5Bobject%20Object%5D&amp;results[]=%5Bobject%20Object%5D&amp;results[]=%5Bobject%20Object%5D&amp;results[]=%5Bobject%20Object%5D&amp;results[]=%5Bobject%20Object%5D&amp;results[]=%5Bobject%20Object%5D&amp;results[]=%5Bobject%20Object%5D&amp;results[]=%5Bobject%20Object%5D&amp;results[]=%5Bobject%20Object%5D&amp;results[]=%5Bobject%20Object%5D&amp;results[]=%5Bobject%20Object%5D&amp;results[]=%5Bobject%20Object%5D&amp;results[]=%5Bobject%20Object%5D&amp;results[]=%5Bobject%20Object%5D&amp;results[]=%5Bobject%20Object%5D&amp;results[]=%5Bobject%20Object%5D&amp;results[]=%5Bobject%20Object%5D&amp;results[]=%5Bobject%20Object%5D&amp;results[]=%5Bobject%20Object

In [124]:
html_title

['search-result__title">Code Card - 151 Booster Pack</span><div class="inventory"',
 'search-result__title">151 Booster Pack</span><div class="inventory"',
 'search-result__title">Charmander - 004/165</span><div class="inventory"',
 'search-result__title">Pidgey</span><div class="inventory"',
 'search-result__title">Grabber</span><div class="inventory"',
 'search-result__title">Dodrio</span><div class="inventory"',
 'search-result__title">151 Ultra-Premium Collection</span><div class="inventory"',
 'search-result__title">Giovanni\'s Charisma - 161/165</span><div class="inventory"',
 'search-result__title">151 Elite Trainer Box</span><div class="inventory"',
 'search-result__title">Blastoise ex - 009/165</span><div class="inventory"',
 'search-result__title">Mew ex - 151/165</span><div class="inventory"',
 'search-result__title">Cycling Road</span><div class="inventory"',
 'search-result__title">Rigid Band</span><div class="inventory"',
 'search-result__title">151 Booster Bundle</span><

In [ ]:
# search-result-count includes the number of results!

In [53]:
p = re.compile("^search-result-count.*results")
p

re.compile(r'^search-result-count.*results', re.UNICODE)